In [62]:
import numpy as np
import random

In [63]:
# Define the grid environment
grid_size = 5
grid = np.zeros((grid_size, grid_size))  # The grid
terminal_states = [(0, 0), (grid_size - 1, grid_size - 1)]  # Terminal states

# Define rewards for each state (positive for terminal states)
goal_state = (grid_size - 1, grid_size - 1)
rewards = np.full((grid_size, grid_size), -1)  # Reward of -1 for each step
rewards[0, 0] = 0
#rewards[grid_size - 1, grid_size - 1] = 0

obstacle = (2, 2)

rewards[goal_state] = 10.0  # Positive reward for reaching the goal
rewards[obstacle] = -10.0  # Large negative reward for hitting the obstacle


In [64]:
# Define possible movements based on actions
actions = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

In [65]:
# Define the initial policy (random actions)
policy = {}
# Initialize the policy randomly for non-terminal states
for i in range(grid_size):
    for j in range(grid_size):
        if (i, j) not in terminal_states and (i, j) != obstacle:  # Avoid terminal and obstacle states
            policy[(i, j)] = random.choice(list(actions.keys()))  # Choose a random action
# Initialize state-value function arbitrarily
state_values = np.zeros((grid_size, grid_size))

In [66]:
# Helper function to get the next state after taking an action
def get_next_state(state, action):
    action_delta = actions[action]  # Convert action string to coordinate change
    next_state = (state[0] + action_delta[0], state[1] + action_delta[1])
    
    # Ensure the next state is within bounds and not an obstacle
    if (0 <= next_state[0] < grid_size) and (0 <= next_state[1] < grid_size):
        if next_state != obstacle:  # If it's not the obstacle, move
            return next_state
    
    # If next_state is an obstacle or out of bounds, return the current state
    return state

In [67]:

def monte_carlo_policy_evaluation(episodes, gamma=1.0, max_steps=100):
    returns = {}  # Dictionary to store returns for each state
    for i in range(grid_size):
        for j in range(grid_size):
            returns[(i, j)] = []  # Initialize empty list of returns for each state

    for _ in range(episodes):
        # Generate an episode
        state = (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))
        episode = []
        steps = 0
        visited_states = set()  # Keep track of visited states
        G = 0  # Initialize return
        
        while state not in terminal_states and state != obstacle and steps < max_steps:
            action = policy[state]
            next_state = get_next_state(state, action)
            reward = rewards[next_state]
            
            episode.append((state, reward))
            state = next_state
            steps += 1
        
        # Compute returns and update state values
        for t in range(len(episode)-1, -1, -1):  # Backward pass through the episode
            state, reward = episode[t]
            G = gamma * G + reward
            
            # First-visit Monte Carlo
            if state not in visited_states:
                visited_states.add(state)
                returns[state].append(G)
                state_values[state] = np.mean(returns[state])  # Update state value with average return


In [68]:
# Monte Carlo Policy Iteration
def monte_carlo_policy_iteration(episodes, gamma=1.0):
    for _ in range(episodes):
        monte_carlo_policy_evaluation(episodes, gamma)
        for i in range(grid_size):
            for j in range(grid_size):
                state = (i, j)
                if state not in terminal_states:
                    # Greedy policy improvement based on state values
                    action_returns = {}
                    for action in actions:
                        next_state = get_next_state(state, action)
                        action_returns[action] = rewards[next_state] + gamma * state_values[next_state]
                    policy[state] = max(action_returns, key=action_returns.get)


In [69]:
# Run Monte Carlo Policy Iteration
monte_carlo_policy_iteration(100)

In [70]:

# Print final results
print("Final State Values:")
print(state_values)
print("Final Policy:")
for i in range(grid_size):
    for j in range(grid_size):
        if (i, j) != obstacle:  # Skip obstacle
            if (i, j) in policy:
                print(f"State ({i},{j}):", policy[(i, j)])
            else:
                print(f"State ({i},{j}): No policy assigned")
        else:
            print(f"State ({i},{j}): obstacle")

Final State Values:
[[ 0.  4.  5.  6.  7.]
 [ 4.  5.  6.  7.  8.]
 [ 5.  6.  0.  8.  9.]
 [ 6.  7.  8.  9. 10.]
 [ 7.  8.  9. 10.  0.]]
Final Policy:
State (0,0): No policy assigned
State (0,1): down
State (0,2): down
State (0,3): down
State (0,4): down
State (1,0): down
State (1,1): down
State (1,2): right
State (1,3): down
State (1,4): down
State (2,0): down
State (2,1): down
State (2,2): obstacle
State (2,3): down
State (2,4): down
State (3,0): down
State (3,1): down
State (3,2): down
State (3,3): down
State (3,4): down
State (4,0): right
State (4,1): right
State (4,2): right
State (4,3): right
State (4,4): No policy assigned
